In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
tf.__version__
import keras
keras.__version__
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle
from PIL import Image

#from keras.models import Sequential
#from keras.layers import Dense,Conv2D,Flatten
#from keras.layers import MaxPooling2D
#from keras.layers import Dropout
#from keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import ReduceLROnPlateau
#from keras.constraints import UnitNorm
#from keras import regularizers
#from keras.layers import UpSampling2D
#from keras.layers import UpSampling3D
#from keras.layers import Conv2DTranspose


In [ ]:
#"../input"
mother_directory = os.getcwd()
os.listdir(mother_directory)
print(mother_directory)

In [ ]:
model = tf.keras.models.load_model("../input/models/neuron_creator_unet.hdf5")

In [ ]:
liste_dir = os.listdir("../input/sartorius-cell-instance-segmentation/test")

In [ ]:
def disect_image(image):
    disect_liste = [] 
    disect_liste.append(image[0:256,0:256,1])
    disect_liste.append(image[0:256,256:512,1])
    disect_liste.append(image[0:256,(512-64):704,1])
    
    disect_liste.append(image[256:512,0:256,1])
    disect_liste.append(image[256:512,256:512,1])
    disect_liste.append(image[256:512,(512-64):704,1])
    
    return disect_liste


def reconcile_image(y_pred):
    liste = list(y_pred)
    ara = np.concatenate((liste[0],liste[1]),axis=1)
    ara = np.concatenate((ara,liste[2][:,64:]),axis=1)
    ara2 = np.concatenate((liste[3],liste[4]),axis=1)
    ara2 = np.concatenate((ara2,liste[5][:,64:]),axis=1)
    fin = np.concatenate((ara,ara2),axis=0)
    return fin


In [ ]:
def RLD(s):
    s=s.split(" ")
    ans=[]
    i=0
    while i<len(s):
        j=0
        n=int(s[i])
        while j<int(s[i+1]):
            ans.append(int(n))
            n=n+1
            j=j+1
        i=i+2
    return ans


def mask_placer(annot,current_image_flattened,RGB = "yes"):
    Annot = RLD(annot)
    if RGB == "yes":
        seed = (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
        for i in Annot:
            current_image_flattened[i,0] = seed[0]
            current_image_flattened[i,1] = seed[1]
            current_image_flattened[i,2] = seed[2]
        
      
        
        
        
        current_image = current_image_flattened.reshape(520,704,3)
        return current_image

    else:
        for i in Annot:
            if i<520*704:
                current_image_flattened[i,0] = 255
                current_image_flattened[i,1] = 255
                current_image_flattened[i,2] = 255
        current_image = current_image_flattened.reshape(520,704,3)
        return current_image

In [ ]:
'''
#Creating_neuron means starting a neuron word on dictionary.
def create_neuron():
    name_neuron = "Neuron_"+str(len(Neuron_ledger.keys()))
    Neuron_ledger[name_neuron] = [[],[]]

def enter_pixel_coord(coordinate,neuron_name):
    Neuron_ledger[neuron_name][0].append(coordinate)
    
def clear_last_ad(neuron_name):
    Neuron_ledger[neuron_name][1] = []

def last_ad_check(neuron_name):
    if Neuron_ledger[neuron_name][1] == (0,0):
        return False
    else:
        return True
    
def update_neuron(coordinate_liste,neuron_name,clear_past = "no"):
    if clear_past == "yes":
        clear_last_ad(neuron_name)
    #for i in range(len(coordinate_liste)):
        #enter_pixel_coord(coordinate_liste[i],neuron_name)
    Neuron_ledger[neuron_name][0].append(coordinate_liste)

    Neuron_ledger[neuron_name][1].append(coordinate_liste)
    

    
#Slicing oject will be 1 X 704 numpy object

def get_slice(image,row_number):
    sliced = image[row_number,:]
    row_number = row_number
    return sliced,row_number

def bright_coordinate_calc(sliced):
    locater = np.arange(1,705)
    located_coord_array = locater * sliced.T
    return located_coord_array.T

def get_lumps(located_coord_array):
    liste_lumps = []
    start = 0
    end = 0
    liste_sample = []
    for i in list(located_coord_array):
        if i == 0 and start == 0:
            continue
        elif i > 0 and len(liste_sample) == 0 :
            liste_sample.append(i)
            start = i
        
        elif i > 0 and len(liste_sample) > 0:
            liste_sample.append(i)
            
        elif i == 0 and start > 0:
            end = liste_sample[-1]
            liste_lumps.append((start,end))
            start = 0
            end = 0
            liste_sample = []
            
    return liste_lumps
#This function gives us the most probable neuron that it would be a part of.
def lump_comparer(new_lump):
    differences_list = []
    for i in Neuron_ledger.keys():
        new_lump_mean = (new_lump[0] + new_lump[1])/2
        new_lump_size = new_lump[-1] - new_lump[0]
        last_ad = (Neuron_ledger[i][1][0][0][1],Neuron_ledger[i][1][0][1][1])
        last_ad_mean = (last_ad[0]+last_ad[1])/2
        last_ad_size = last_ad[-1] - last_ad[0]
        diff = abs(last_ad_mean - new_lump_mean)
        differences_list.append(diff)
    
    minimum_diff = min(differences_list)
    if minimum_diff < 30:
        neuron_index = differences_list.index(minimum_diff)
        neuron_continum = list(Neuron_ledger.keys())[neuron_index]
    else:
        neuron_continum  = "new_neuron"
        
    return neuron_continum
        
def whole_lump_neuron_match(liste_lumps):
    for lump in liste_lumps:
        neuron_continum = lump_comparer(lump)
        if neuron_continum == "new_neuron":
            create_neuron()
            coordinate_liste = list((lump[0],lump[1]))
            for k in range(len(coordinate_liste)):
                coordinate_liste[k] = (row_number,float(coordinate_liste[k]))
            update_neuron(coordinate_liste,list(Neuron_ledger.keys())[-1],clear_past="yes")
            
        else:
            coordinate_liste = list((lump[0],lump[1]))
            for k in range(len(coordinate_liste)):
                coordinate_liste[k] = (row_number,float(coordinate_liste[k]))
            update_neuron(coordinate_liste,neuron_continum,clear_past="yes")
            
            
def neuron_finished_matches(row_number):
    for i in Neuron_ledger.keys():
        last_ad = (Neuron_ledger[i][1][0][0][0],Neuron_ledger[i][1][0][1][0])
        if last_ad[0] < row_number:
            Neuron_ledger[i][1][0] = [(0,0),(0,0)]
                    
        
            

def lump_ledger_link(liste_lumps,row_number):
    if len(Neuron_ledger.keys()) == 0:
        for i in range(len(liste_lumps)):
            create_neuron()
            #coordinate_liste = list(np.arange(liste_lumps[i][0],liste_lumps[i][1]+1))
            coordinate_liste = list((liste_lumps[i][0],liste_lumps[i][1]))
            for k in range(len(coordinate_liste)):
                coordinate_liste[k] = (row_number,float(coordinate_liste[k]))

           
            update_neuron(coordinate_liste,list(Neuron_ledger.keys())[-1],clear_past="yes")
            
    elif len(Neuron_ledger.keys()) > 0:
        
        whole_lump_neuron_match(liste_lumps)
        
        
def find_edges(Neuron_name):
    liste = Neuron_ledger[Neuron_name][0]
    fark = 0
    which_fark = 0
    counter = 0
    for i in liste:
        if i[1][1] - i[0][1] > fark:
            fark = i[1][1] - i[0][1]
            which_fark = counter
        
        counter += 1
    return fark,len(liste),which_fark,liste

def vision_applier(sample):
    modified_im = sample.copy()
    for i in Neuron_ledger.keys():
        fark,length_liste,which_fark,liste = find_edges(i)
        modified_im[int(liste[0][0][0]):int(liste[-1][0][0]),int(liste[which_fark][0][1])] = 1
        modified_im[int(liste[0][0][0]):int(liste[-1][0][0]),int(liste[which_fark][1][1])] = 1

        modified_im[int(liste[0][0][0]),int(liste[which_fark][0][1]):int(liste[which_fark][1][1])] = 1
        modified_im[int(liste[-1][0][0]),int(liste[which_fark][0][1]):int(liste[which_fark][1][1])] = 1
    return modified_im

def encoding_pixels(pixel_list):
    coding = ""
    begin = pixel_list[0]
    counter = 0
    current = begin
    for pixel in pixel_list:
        
        if pixel == pixel_list[0]:
            counter += 1
            continue
        elif current+1 == pixel:
            current = pixel
            counter += 1
            
        elif current +1 != pixel:
            coding =  coding + str(begin) + " " + str(counter) + " "
            begin = pixel
            counter = 1
            current = begin
            
     

    code = coding[:len(coding)-1]
            
    return code

def encoding_pixels(pixel_list):
    if len(pixel_list) < 10:
        return 0
    else:
        coding = ""
        begin = pixel_list[0]
        counter = 0
        current = begin
        for pixel in pixel_list:

            if pixel == pixel_list[0]:
                counter += 1
                continue
            elif current+1 == pixel:
                current = pixel
                counter += 1

            elif current +1 != pixel:
                coding =  coding + str(begin) + " " + str(counter) + " "
                begin = pixel
                counter = 1
                current = begin



        code = coding[:len(coding)-1]

        return code'''
            
        
     #last   
def encoding_pixels(pixel_list):
    if True:
        coding = ""
        begin = pixel_list[0]
        counter = 0
        current = begin
        for pixel in pixel_list:

            if pixel == pixel_list[0]:
                counter += 1
                continue
            elif current+1 == pixel:
                current = pixel
                counter += 1

            elif current +1 != pixel:
                coding =  coding + str(begin) + " " + str(counter) + " "
                begin = pixel
                counter = 1
                current = begin



        code = coding[:len(coding)-1]

        return code
            



In [ ]:
'''def encoding(i,max_line):
    first_coord = (i[2])*max_line + i[0]
    second_coord = (i[2])*max_line + i[1]
        
    final =str(int(first_coord))+ " " +  str(int(second_coord-first_coord+1))
    return final

def encoding(i,max_line):
        first_coord = (i[0][0])*max_line + i[0][1]
        second_coord = (i[1][0])*max_line + i[1][1]
        
        final =str(int(first_coord))+ " " +  str(int(second_coord-first_coord+1))
        return final'''

In [ ]:
'''#Bedir code
submission = pd.DataFrame(columns = ["id","predicted"])
counter = 0
for im in liste_dir:
    image = cv2.imread("../input/sartorius-cell-instance-segmentation/test/"+im)
    
    barsak = disect_image(image)
    test = np.array(barsak).reshape(6,256,256,1)
    y_pred = model.predict(test)
    fin = reconcile_image(y_pred)
    #plt.imshow(fin)
    fin.shape
    
    sample = fin.copy()
    flattened = sample.flatten()

    for i in range(len(flattened)):
        if flattened[i] > 0.5:
            flattened[i] = 1.0
        else:
            flattened[i] = 0.0
    sample = flattened.reshape(512,704,1)
    
    #plt.imshow(sample)
    
    
    #Neuron_structure = [[],[]]
    Neuron_ledger = {}

    for i in range(0,512):
        sliced,row_number = get_slice(sample,i)
        located_coord_array = bright_coordinate_calc(sliced)
        liste_lumps = get_lumps(located_coord_array)
        lump_ledger_link(liste_lumps,row_number)
        neuron_finished_matches(row_number)
        
    #result = vision_applier(sample)
    
    #plt.imshow(result)
    
    liste = []
    for name in Neuron_ledger.keys():

        altogether = ""
        counter = 0
        for i in Neuron_ledger[name][0]:
            if counter == 0:
                altogether = encoding(i,704)
                counter += 1
            else:
                coord = encoding(i,704)
                altogether = altogether + " " + coord

        if len(altogether) > 10:
            liste.append(altogether)
            
    name = []
    for i in range(len(liste)):
        name.append(im[:-4]) 
            
    #print(len(liste))
    if counter == 0:
        submission["id"] = name
        submission["predicted"] = liste
        counter += 1
    else:
        new = pd.DataFrame(columns = ["id","predicted"])
        new["id"] = name
        new["predicted"] = liste
        submission = pd.concat([submission,new],axis=0)
        '''

In [ ]:
submission = pd.DataFrame(columns = ["id","predicted"])
counter = 0

for im in liste_dir:
    image = cv2.imread("../input/sartorius-cell-instance-segmentation/test/"+im)
    
    barsak = disect_image(image)
    test = np.array(barsak).reshape(6,256,256,1)
    y_pred = model.predict(test)
    fin = reconcile_image(y_pred)
    #plt.imshow(fin)
    #fin.shape
    
    sample = fin.copy()
    flattened = sample.flatten()

    flattened = flattened*255
    for i in range(len(flattened)):
        if flattened[i] > 50:
            flattened[i] = 255
        else:
            flattened[i] = 0
    sample = flattened.reshape(512,704,1)
    
    #plt.imshow(sample)
    
    
    ret1,tresh = cv2.threshold(sample,0,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(tresh,cv2.MORPH_OPEN,kernel,iterations=2)
    sure_bg = cv2.dilate(opening,kernel,iterations=10)
    #plt.imshow(sure_bg)
    opening = opening.astype(np.uint8())
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    #plt.imshow(dist_transform)
    ret2,sure_fg = cv2.threshold(dist_transform,0.25*dist_transform.max(),255,0)
    #plt.imshow(sure_fg)
    
    sure_fg = np.uint8(sure_fg)
    sure_bg = np.uint8(sure_bg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret3,markers = cv2.connectedComponents(sure_fg)
    #plt.imshow(markers,cmap="jet")
    markers = markers+10
    markers[unknown==255] = 0
    #plt.imshow(markers,cmap="gray")
    sample1 = np.concatenate((sample,sample),axis=2)
    sample1 = np.concatenate((sample1,sample),axis=2)
    sample1.shape
    sample1 = np.uint8(sample1)
    markers = cv2.watershed(sample1,markers)
    #plt.imshow(markers)
    
    labels = markers.flatten()
    dicti = {}
    for label in labels:
        if (str(label) not in dicti.keys()) and (label != -1) and (label != 10):
            dicti[str(label)] = []
            
    
    for i in range(len(labels)):
        if labels[i] != -1 and labels[i] != 10 :
            dicti[str(labels[i])].append(i)
        
    '''picti = np.zeros((512*704))
    counter = 0
    for k in dicti.keys():
        if len(dicti[k]) > 10:
            hedef = dicti[k]
            for i in hedef:
                if picti[i] != 0:
                    print("occupied")
                else:
                    picti[i] = 1
                    counter += 1
    
    #plt.imshow(picti.reshape(512,704))'''
                
                    

    
    liste = []
    for name in dicti.keys():
        if len(dicti[name]) > 10:
            code = encoding_pixels(dicti[name])
            liste.append(code)
             
    
    
    name = []
    for i in range(len(liste)):
        name.append(im[:-4]) 
        
        
    
    print(len(liste))
    if counter == 0:
        submission["id"] = name
        submission["predicted"] = liste
        counter += 1
    else:
        new = pd.DataFrame(columns = ["id","predicted"])
        new["id"] = name
        new["predicted"] = liste
        submission = pd.concat([submission,new],axis=0)

        
        

    
        

In [ ]:
submission.info()

In [ ]:
submission.head()

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv",index=False)